In [1]:
import sys
sys.path.append('..')

### Set OpenAI key 

In [2]:
import os
import configparser

config = configparser.ConfigParser()
config.read('../../.secrets.ini')
OPENAI_API_KEY = config['OPENAI']['OPENAI_API_KEY']
YOUTUBE_KEY = config['YOUTUBE']['YOUTUBE_API_KEY']
NAVER_CLIENT_ID = config['NAVER']['NAVER_CLIENT_ID']
NAVER_CLIENT_SECRET = config['NAVER']['NAVER_CLIENT_SECRET']
GOOGLE_SEARCH_KEY = config['GOOGLE']['GOOGLE_API_KEY']
CSE_ID = config['GOOGLE']['CSE_ID']
SERPAPI_API_KEY = config['SERPAPI']['SERPAPI_API_KEY']


os.environ.update({'OPENAI_API_KEY': OPENAI_API_KEY})
os.environ.update({'YOUTUBE_KEY': YOUTUBE_KEY})
os.environ.update({'NAVER_CLIENT_ID': NAVER_CLIENT_ID})
os.environ.update({'NAVER_CLIENT_SECRET': NAVER_CLIENT_SECRET})
os.environ.update({'GOOGLE_SEARCH_KEY': GOOGLE_SEARCH_KEY})
os.environ.update({'CSE_ID': CSE_ID})
os.environ.update({'SERPAPI_API_KEY': SERPAPI_API_KEY})

### Get tools

In [3]:
# %load_ext autoreload
# %autoreload 2
%reload_ext autoreload

In [4]:
from importlib import reload
import tools
reload(tools)

<module 'tools' from 'c:\\Users\\chois\\Desktop\\OpenAI_SKT\\openai_skt\\tutorials\\..\\tools\\__init__.py'>

In [5]:
from tools import DatabaseTool, SearchTool, SearchByURLTool, TimeTool
from models.llm.chain import UnifiedSummaryChunkChain

In [6]:
summary_chunk_template = '''
I want you to answer the question from the given documentation. 
There may be a lot of information in a given document that is not relevant to your question, but you should try to find as much relevant information as possible.
Be as diligent as possible in returning your answer based on your knowledge and documentation.
If there is no answer in the document, return "No useful information".

Document: {document}
Question: {question}
Answer:
'''

In [7]:
summary_chunk_chain = UnifiedSummaryChunkChain(summary_chunk_template=summary_chunk_template, input_variables=["question", "document"], verbose=True)
search_by_url_tool = SearchByURLTool()

In [8]:
from langchain.agents import Tool
from langchain.utilities import SerpAPIWrapper

# You can create the tool to pass to an agent
search = SerpAPIWrapper(params={
        "engine": "google",
        "google_domain": "google.com",
        "gl": "kr",
        "hl": "ko",
        })
search_tool = Tool(
    name="search_tool",
    description="A tool to search on internet.",
    func=search.run,
)

In [9]:
database_tool = DatabaseTool(summary_chunk_chain=summary_chunk_chain)
search_tool = SearchTool(summary_chunk_chain=summary_chunk_chain, search_by_url_tool=search_by_url_tool)
time_tool = TimeTool()

In [10]:
tools = [database_tool, search_tool, time_tool]

In [11]:
from database import CustomEmbedChain
embed_chain = CustomEmbedChain()

In [12]:
from database import DataBase

In [13]:
# database = DataBase.load(database_path='./user/test_2/database.json', embed_chain=embed_chain) # optimize: 5m22s # 4m18s 3m4s 3m2s

All data from https://www.business2community.com/kr/cryptocurrency/bitcoin-price-prediction already exists in the database.
All data from https://www.btcc.com/ko-KR/academy/research-analysis/bitcoin-price-forecast already exists in the database.
All data from https://www.bonmedia.kr/news/articleView.html?idxno=410 already exists in the database.
All data from https://conveyvalue.tistory.com/entry/%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8-%EA%B0%80%EA%B2%A9-%EC%98%88%EC%B8%A1-%EC%B5%9C%EA%B7%BC-%EB%8F%99%ED%96%A5%EA%B3%BC-%EB%AF%B8%EB%9E%98-%EC%A0%84%EB%A7%9D already exists in the database.
All data from https://www.dailysecu.com/news/articleView.html?idxno=143593 already exists in the database.
All data from https://www.sisamagazine.co.kr/news/articleView.html?idxno=487081 already exists in the database.
All data from https://www.ajunews.com/view/20230213184909147 already exists in the database.
All data from https://www.bonmedia.kr/news/articleView.html?idxno=409 already exists in the data

In [15]:
import pickle

with open("database_save_path.pkl", "wb") as f:
    pickle.dump(database, f)

with open("database_save_path.pkl", "rb") as f:
    database = pickle.load(f)
database.set_embed_chain(embed_chain)


### Set Agent

In [16]:
from models.llm.qna_agent import QnAAgent

In [17]:
qna_agent = QnAAgent(tools=tools, verbose=True, qna_prompt_path='../../openai_skt/models/templates/qna_prompt_template.txt', model='gpt-4')

In [18]:
database_tool = DatabaseTool(summary_chunk_chain=summary_chunk_chain)
database_tool.set_database(database=database)
tools = [database_tool, search_tool, time_tool]

In [20]:
answer = qna_agent.run(tools=tools, question="너 이름이 뭐야?", qna_history=[])



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. Use a variety of tools to answer the questions. You have access to the following tools:

database: A tool to get data with from database. The input consists of a 'query' and a 'question', where 'query' is the search query and 'question' is the information you want to get. For example, {'query': 'bitcoin price history', 'question': 'what was the price of bitcoin in 2022?'} would be the input if you want to know the price of bitcoin in 2022. Input must contain both a query and a question.
search_tool: A tool to search on internet. The input consists of a 'query' and a 'question', where 'query' is the search query and 'question' is the information you want to get. For example, {'query': 'bitcoin price history', 'question': 'what was the price of bitcoin in 2022?'} would be the input if you want to know the price of bitcoin in 2022. Input must 